### LIBRARIES

In [2]:
# Deal with data
import pandas as pd
import json
from glob import glob

# Deal with visualization
import matplotlib.pyplot as plt

# Deal with time
import datetime

## Github API

A API do GitHub utiliza principalmente o protocolo HTTP, onde as operações são realizadas através de URLs e métodos HTTP, como GET, POST, PUT e DELETE. Os dados são geralmente retornados em formato JSON, tornando-o fácil de processar e interpretar.

In [ ]:
data = glob('./analytics-raw-data/GitHub_API-fga-eps-mds-*.json')
# print(data)

#### Unmarshall json

In [ ]:
def unmarshall(json_path: str) -> dict:
    with open(json_path) as json_file:
        json_obj = json.load(json_file)
    return json_obj

### Cálculo dos dados

- Carrega dados dos arquivo JSON e armazena-os na variável data.
- Processa os dados do JSON para calcular e obter informações relevantes sobre os tempos de feedback de cada execução de workflow, separando-os em listas de feedback_times, authors_feedback_times, e ci_feedback_times.
- Calcula o tempo médio de feedback (ci_feedback_time) considerando todas as execuções de workflow.
- Calcula o tempo médio de feedback de cada autor e armazena essas informações em authors_avg_feedback_times.
- Processa novamente os dados do JSON para contar o número de execuções bem-sucedidas (success_count) e falhas (failure_count) dos workflows.

In [ ]:
tabela_total = pd.DataFrame()

# Processar os dados do JSON e obter as informações relevantes
success_count = 0
failure_count = 0

for json_path in data:
    dados = unmarshall(json_path)
    # Create a list to store the data for the table
    table_data = []
    # Processar os dados do JSON e obter as informações relevantes
    for run in dados["workflow_runs"]:
        updated_at = datetime.datetime.strptime(run["updated_at"], "%Y-%m-%dT%H:%M:%SZ")
        created_at = datetime.datetime.strptime(run["created_at"], "%Y-%m-%dT%H:%M:%SZ")
        feedback_time = (updated_at - created_at).total_seconds()
        
        # informações relevantes
        workflow_id = run["id"]
        conclusion = run["conclusion"]
        author = run["actor"]["login"]
        
        # Adiciona os dados na tabela
        table_data.append({
            "Workflow_run ID": workflow_id,
            "Conclusion": conclusion,
            "Author": author,
            "Created at": created_at,
            "Updated at": updated_at,
            "Feedback Time": feedback_time
        })
        
        
    # Cria um dataframe com o pandas
    df = pd.DataFrame(table_data)

    
    # table_data_global.append(df)
    
    # Processar os dados do JSON e obter as informações relevantes
    feedback_times = []
    authors_feedback_times = {}
    ci_feedback_times = []

    for run in df.index:
        updated_at = df["Updated at"][run]
        created_at = df["Created at"][run]
        feedback_time = (updated_at - created_at).total_seconds()
        feedback_times.append(feedback_time)
        
    # Calcular o CI FeedBack Time
    ci_feedback_time = sum(feedback_times) / len(feedback_times)
    df["Ci Feedback Time"] = ci_feedback_time
    
    # Tabela com todas as linhas
    tabela_total = pd.concat([tabela_total, df])
    # print('Tabela por dia')
    # display(df)

    for run in dados["workflow_runs"]:
        conclusion = run["conclusion"]
        if conclusion == "success":
            success_count += 1
        elif conclusion == "failure":
            failure_count += 1
                
print('Tabela com todas as linhas')
display(tabela_total)



# Gerar gráficos

- Gráfico de linhas: Mostra o "CI FeedBack Time" (tempo de feedback do CI) em relação à "Data de Atualização" das execuções de workflow.
- Gráfico de barras simples: Exibe o "Tempo Médio de Feedback do CI", representando o desempenho médio geral do CI.
- Gráfico de pizza: Mostra a "Quantidade de Workflows com Sucesso e Falha" em relação ao total de workflows executados, indicando a proporção de workflows que tiveram sucesso e falha.

In [ ]:
# Gerar gráfico com o CI FeedBack Time por data de atualização

temp = tabela_total[['Created at', 'Ci Feedback Time']]
dates = temp.groupby(['Created at']).mean().reset_index()  # Calcula a média dos valores por data
dates['Created at'] = pd.to_datetime(dates['Created at']) 
print(dates)


# dates = [datetime.datetime.strptime(run["updated_at"], "%Y-%m-%dT%H:%M:%SZ") for run in data["workflow_runs"]]
plt.figure(figsize=(20, 10))
plt.plot(range(len(dates)), dates['Ci Feedback Time'], marker='o', color='b', label='Feedback Time')
plt.xlabel("Created at")
plt.ylabel("CI FeedBack Time")
plt.title("CI FeedBack Time por Data de Criação")
plt.xticks(range(len(dates)), dates['Created at'].dt.strftime('%Y-%m-%d'), rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()  # Ajusta a disposição dos elementos para evitar sobreposições
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
bars =  plt.bar(range(len(dates)), dates['Ci Feedback Time'], color='salmon', label='Feedback Time')
plt.xlabel("Created at")
plt.ylabel("CI FeedBack Time")
plt.title("CI FeedBack Time por Data de Criação")
plt.xticks(range(len(dates)), dates['Created at'].dt.strftime('%Y-%m-%d'), rotation=45)  # Exibe as datas formatadas no eixo X
plt.legend()
# Adiciona as labels em cada barra
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, str(height), ha='center', va='bottom')

plt.tight_layout()  # Ajusta a disposição dos elementos para evitar sobreposições
plt.show()

# # Gerar gráfico com o tempo médio do CI
# plt.figure(figsize=(20, 10))
# plt.bar(["CI"], [ci_feedback_time], color='salmon')
# plt.xlabel("CI")
# plt.ylabel("Tempo Médio de Feedback")
# plt.title("Tempo Médio de Feedback do CI")
# plt.text(0, ci_feedback_time, f'{round(ci_feedback_time, 2)} segundos', ha='center', va='bottom', fontweight='bold')
# plt.tight_layout()
# plt.show()

In [ ]:
# Gerar gráfico com a quantidade de workflows com sucesso e falha
labels = ["Success", "Failure"]
sizes = [success_count, failure_count]
plt.figure(figsize=(20, 10))
colors = ['lightgreen', 'crimson']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors)
plt.title("Quantidade de Workflows_runs com Sucesso e Falha")
plt.legend(labels, loc='best')
plt.tight_layout()
plt.show()